In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
%matplotlib inline

In [3]:
mushroom = pd.read_table('connect/connect.dat', sep='\s+', header=None)

In [4]:
print('columns of data:', len(mushroom.columns))
print('rows of data:', len(mushroom.index))
print('items:', 119)
print('bits of String:', math.ceil(np.log2(np.log2(len(mushroom.index)))))

columns of data: 43
rows of data: 67557
items: 119
bits of String: 5


In [5]:
def getStream(rawdata):
    temp = list()
    n = len(rawdata.index)
    m = len(rawdata.columns)
    for i in range(n):
        for j in range(m):
            temp.append([rawdata.iloc[i,j], i])
    data = np.array(temp, dtype = int)
    return data

In [6]:
stream = getStream(mushroom)
len(stream)

2904951

In [7]:
def getComplete(stream):
    complete = dict()
    for i in range(len(stream)):
        item = stream[i][0]
        record = stream[i][1]
        if item not in complete.keys():
            complete[item] = set()
        complete[item].add(record)
    return complete

def getRealJC(complete, setA_index, setB_index):
    setA = complete[setA_index]
    setB = complete[setB_index]
    return len(setA&setB)/len(setA|setB)
    
complete = getComplete(stream)

In [8]:
k = 200

In [9]:
def getSketch(stream):
    global k
    sketch = dict()
    for i in range(len(stream)):
        item = stream[i][0]
        record = stream[i][1]
        if item not in sketch.keys():
            sketch[item] = - np.ones((k,1+1),dtype='int8')
        for j in range(k):
            random.seed(record+j)
            mu = math.floor(- np.log2(random.uniform(0, 1)))
            if mu == sketch[item][j][1]:
                sketch[item][j][0] = 0
            elif mu > sketch[item][j][1]:
                sketch[item][j][0] = 1
                sketch[item][j][1] = mu
    return sketch

In [10]:
sketch = getSketch(stream)


In [11]:
def getEstimateJC(sketch):
    global k
    jaccards = dict()
    items = list(sketch.keys())
    for i in range(len(items) - 1):
        for j in range(i + 1, len(items)):
            a = items[i]
            b = items[j]
            flag = 0
            for h in range(k):
                x1 = x2 = 0 
                if sketch[a][h][1] != sketch[b][h][1]:
                    x1 = 1
                    if sketch[a][h][1] > sketch[b][h][1]:
                        x2 = sketch[a][h][0]
                    elif sketch[a][h][1] < sketch[b][h][1]:
                        x2 = sketch[b][h][0]
                flag += x1 * x2
            jc = 1 - flag / (k * 0.7213)
            jaccards[(a, b)] = jc
    return jaccards

In [12]:
jaccards = getEstimateJC(sketch)


In [13]:
def evaluate(jaccards, complete):
    tp = fp = fn = tn = 0
    for i,j in jaccards.items():
        setA_index = i[0]
        setB_index = i[1]
        realJC = getRealJC(complete, setA_index, setB_index)
        estimateJC = j
        if realJC > 0.8 and estimateJC >0.8:
            tp += 1
        elif realJC > 0.8 and estimateJC <= 0.8:
            fn += 1
        elif realJC <= 0.8 and estimateJC > 0.8:
            fp += 1
        elif realJC <= 0.8 and estimateJC <= 0.8:
            tn += 1
    print(tp, fp, fn ,tn)
    recall = tp / (tp + fn)
    print('reacall:', recall)
    precision = tp / (tp + fp)
    print('precision:',precision)


In [14]:
evaluate(jaccards, complete)

275 326 49 7606
reacall: 0.8487654320987654
precision: 0.45757071547420963
